# Accounting Logic

In [1]:
import random

random.seed(0)

In [2]:
global assets
global data_source
global current_prices 

assets = ['asset1', 'asset2']
data_source = (lambda assets: ({asset: random.random() for asset in assets} for i in range(1000000)))(assets)
current_prices = next(data_source)
print('current_prices: ', current_prices)

current_prices:  {'asset1': 0.8444218515250481, 'asset2': 0.7579544029403025}


In [47]:
class Portfolio:
    def __init__(self, assets: list, init_cash=1_000_000, acc_type="cash", init_margin=1.0):
        self.ledger = {asset: 0. for asset in assets}
        self.init_cash = init_cash
        self.type = acc_type
        
        self._cash = init_cash
        self.initial_margin = init_margin
        self._used_margin = 0.
        self._borrowed_cash = 0.
        self._available_margin = init_cash
        
    def __repr__(self):
        return f"""cash: {self._cash} 
                \nbalance: {self.balance}
                \nborrowed cash: {self._borrowed_cash}
                \nborrowed asset value: {self.borrowed_asset_value}
                \nequity: {self.equity} 
                \nledger: {str(self.ledger)}"""
    
    
    @property
    def cash(self):
        return self._cash 
    
    @property
    def borrowed_asset_value(self):
        return sum((current_prices[k]*v for k, v in self.ledger.items() if v < 0.))
    
    @property
    def borrowed_cash(self):
        return self._borrowed_cash
    
    @property
    def assets_value(self):
        return sum((current_prices[k]*v for k, v in self.ledger.items()))
    
    @property
    def equity(self):
        return self._cash + self.assets_value - self._borrowed_cash
        
    @property
    def balance(self):
        return self._cash 
    
    @property
    def pnl(self):
        return 
        
    @property
    def purchasing_power(self):
        return self._cash / self.initial_margin

    def transaction_cash(self, asset_name, units):
        amount = current_prices[asset_name] * units
        self._cash -= amount
        self.ledger[asset_name] += units
    
    def transaction_margin(self, asset_name, units):
        amount = current_prices[asset_name] * units
        used_margin = amount * self.initial_margin
        self._borrowed_cash += amount * (1-self.initial_margin)
        self._cash -= used_margin
        self._used_margin += used_margin
        self.ledger[asset_name] += units
        
    def order(self, asset_name: str, units: float):
        if self.type == "cash":
            self.transaction_cash(asset_name, units)
        elif self.type == "margin":
            self.transaction_margin(asset_name, units)
        return self

In [48]:
port = Portfolio(assets, acc_type="margin", init_margin=0.2)

In [49]:
port.order('asset1', 1000.), current_prices

(cash: 999818.3774229609 
                 
 balance: 999818.3774229609
                 
 borrowed cash: 726.4903081562682
                 
 borrowed asset value: 0
                 
 equity: 1000000.0 
                 
 ledger: {'asset1': 1000.0, 'asset2': 0.0},
 {'asset1': 0.9081128851953352, 'asset2': 0.5046868558173903})

In [50]:
def buy_test():
    cash = init_cash = 1_000_000
    port = Portfolio(assets, init_cash = init_cash, acc_type="cash")
    price = current_prices[assets[0]]
    units = 1000
    port.order(assets[0], units)
    cost = price * units
    cash -= cost
    borrowed_cash = 0.
    usedMargin = cost
    new_price = price
    equity = init_cash + units*(new_price-price)
    assert cash == port.cash
    assert borrowed_cash == port.borrowed_cash
    assert equity == port.equity

In [67]:
def transaction_test(buy=True, margin=1.):
    assert 0. <= margin <= 1., "margin (required initial) must be between 0 and 1"
    cash = init_cash = 1_000_000
    acc_type = "margin" if margin < 1. else "cash"
    port = Portfolio(assets, init_cash = init_cash, acc_type=acc_type)
    global current_prices
    num_trans=4
    for i in  range(num_trans):
        current_prices = next(data_source)
        price = current_prices[assets[0]]
        units = 1000 if buy else -1000

        # port internal accounting
        port.order(assets[0], units)

        # explicit accounting
        cost = price * units
        cash -= cost
        borrowed_cash = 0.
        usedMargin = cost
        new_price = price
        equity = init_cash + units*(new_price-price)
        
    assert cash == port.cash
    assert borrowed_cash == port.borrowed_cash
    assert equity == init_cash, "No price change and no transaction - eq should be init_cash"
    
from functools import partial
buy_test = partial(transaction_test, buy=True)
sell_test = partial(transaction_test, buy=False)

In [68]:
buy_test()
buy_test(margin=0.2)
sell_test()
sell_test(margin=0.2)